In [1]:
import numpy as np
import scipy
import daqp
from ctypes import * 
import ctypes.util
from sympy import Matrix

from generate_mpqp_v2 import generate_qp
from generate_mpqp_v2 import generate_rhs
#from self_impl_daqp_new import daqp_self
from self_implement_daqp import daqp_self

In [79]:
import self_implement_daqp  
import importlib  
importlib.reload(self_implement_daqp)  
from self_implement_daqp import daqp_self  

solution = []
for i in range(1000):
    
    H,f,F,A,b,B = generate_qp(2,5)
    ftot,btot = generate_rhs(f,F,b,B)
    sense = np.zeros(btot.shape, dtype=np.int32)
    blower = np.array([-np.inf for i in range(btot.shape[0])])

    print(H.shape,ftot.shape,A.shape,btot.shape,blower.shape,sense.shape)
    blower = np.array([-np.inf for i in range(btot.shape[0])])
    x,fval,exitflag,info = daqp.solve(H,ftot,A,btot,blower,sense)
    print("Optimal solution:")
    print(x)
    print("Exit flag:",exitflag)
    print("Info:",info)

    x_star, lam_star, W, iter = daqp_self(H,ftot,A,btot,sense,[])  
    print("final outcome", x_star, lam_star,W, "iterations", iter)
    if x.any() != x_star.any():
        solution.append(1)
    else:
        solution.append(0)    

print(np.sum(solution))

(2, 2) (2,) (5, 2) (5,) (5,) (5,)
Optimal solution:
[-1.77042163  1.26427847]
Exit flag: 1
Info: {'solve_time': 2.3e-06, 'setup_time': 3.9e-06, 'iterations': 5, 'nodes': 1, 'lam': array([0.        , 6.14163348, 0.        , 1.58662264, 0.        ])}
final outcome [-1.77042163  1.26427847] [0.         6.14163348 0.         1.58662264 0.        ] [np.int64(1), np.int64(3)] iterations 5
(2, 2) (2,) (5, 2) (5,) (5,) (5,)
Optimal solution:
[1.12582277 1.02753989]
Exit flag: 1
Info: {'solve_time': 1.6e-06, 'setup_time': 4.4e-06, 'iterations': 2, 'nodes': 1, 'lam': array([0.        , 0.        , 0.        , 0.17178915, 0.        ])}
final outcome [1.12582277 1.02753989] [0.         0.         0.         0.17178915 0.        ] [np.int64(3)] iterations 2
(2, 2) (2,) (5, 2) (5,) (5,) (5,)
Optimal solution:
[-0.81404404 -1.85645352]
Exit flag: 1
Info: {'solve_time': 9e-07, 'setup_time': 2.7e-06, 'iterations': 2, 'nodes': 1, 'lam': array([0.        , 0.        , 0.        , 0.97869028, 0.        ])

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
import self_implement_daqp  
import importlib  
importlib.reload(self_implement_daqp)  
from self_implement_daqp import daqp_self  

H,f,F,A,b,B = generate_qp(2,5)
ftot,btot = generate_rhs(f,F,b,B)
sense = np.zeros(btot.shape, dtype=np.int32)
blower = np.array([-np.inf for i in range(btot.shape[0])])
print(H.shape,ftot.shape,A.shape,btot.shape,blower.shape,sense.shape)
blower = np.array([-np.inf for i in range(btot.shape[0])])
x,fval,exitflag,info = daqp.solve(H,ftot,A,btot,blower,sense)
print("Optimal solution:")
print(x)
print("Exit flag:",exitflag)
print("Info:",info)

x_star, lam_star, W, iter = daqp_self(H,ftot,A,btot,sense,[])  
print("final outcome", x_star, lam_star,W, "iterations", iter)
# x_star, lam_star, W, iter = daqp_self(H,ftot,A,bupper,sense,[])
# print(f"x*={x_star}, lam*={lam_star},W={W}, #iterations: {iter}")

1.8307981033312308
(2, 2) (2,) (5, 2) (5,) (5,) (5,)
Optimal solution:
[0.00309634 0.1689497 ]
Exit flag: 1
Info: {'solve_time': 2.5e-06, 'setup_time': 5.1e-06, 'iterations': 2, 'nodes': 1, 'lam': array([0.        , 0.70199548, 0.        , 0.        , 0.        ])}
final outcome [0.00309634 0.1689497 ] [0.         0.70199548 0.         0.         0.        ] [np.int64(1)] iterations 2


In [83]:
print(H.shape,ftot.shape,A.shape,btot.shape,blower.shape,sense.shape)
print(H)
blower = np.array([-np.inf for i in range(btot.shape[0])])
x,fval,exitflag,info = daqp.solve(H,ftot,A,btot,blower,sense)
print("Optimal solution:")
print(x)
print("Exit flag:",exitflag)
print("Info:",info)

# x_star, lam_star, W, iter = daqp_self(H,ftot,A,btot,sense,[])  
# print("final outcome", x_star, lam_star,W, "iterations", iter)

# x_star, lam_star, W, iter = daqp_self(H,ftot,A,bupper,sense,[])
# print(f"x*={x_star}, lam*={lam_star},W={W}, #iterations: {iter}")

(2, 2) (2,) (5, 2) (5,) (5,) (5,)
[[ 2.97068507 -2.2419259 ]
 [-2.2419259   1.69194365]]
Optimal solution:
[0.19669231 0.10126967]
Exit flag: 1
Info: {'solve_time': 3.3e-06, 'setup_time': 8.6e-06, 'iterations': 3, 'nodes': 1, 'lam': array([0.        , 1.74365037, 1.86833335, 0.        , 0.        ])}


In [62]:
def is_invertible(A):
    return A.shape[0] == A.shape[1] and np.linalg.matrix_rank(A) == A.shape[0]

def fix_component(lam,lam_star,mu, W, B, p):
    print("run: fix component")
    print("B in fixed component",B)
    print("lam",lam)
    print("p",p)
    print("-lam[B]/p[B]",-lam[B]/p[B])
    j = np.argmin(-lam[B]/p[B])
    print("j",j)
    W_new = [int(i) for i in W if int(i)!=int(B[j])]
    print("W",W)
    print("new W", W_new)
    print("lam",lam)
    lam = lam - (lam[B[j]]/p[B[j]])*p
    lam[np.abs(lam) < 1e-12] = 0
    print("new lam",lam)
    #lam_star[B[j]] = 0
    lam_star = np.copy(lam)
    print("new lam_star",lam_star)
    return lam,lam_star,mu, W_new          


In [80]:
def daqp_self(H,f,A,b,sense,W):
    # transform matrices into M,d,v,R
    try:
        R = np.linalg.cholesky(H, upper = True)
    except:
        print("Invalid input")
        raise ValueError("Invalid input")
    M = A @ np.linalg.inv(R)
    print("shape of M", M.shape)
    row_norm = np.linalg.norm(M, axis=1, keepdims=True, ord=2)
    print("row_norm",row_norm)
    print("M before", M)
    M_normed = M / row_norm  # Normalize M
    print("M after",M_normed)
    v = np.linalg.inv(R).T @f
    d = b.reshape(-1) + M @ v
    print("d before", d)
    d_normed = d / row_norm.reshape(-1)
    print("d after",d_normed)
    M = np.copy(M_normed)
    d = np.copy(d_normed)
    print("initial values")
    print("R",R)
    print("M", M)
    print("v", v)
    print("d",d)
    print("---------")
    # initial values
    lam = np.array([0 for i in range(len(b))],dtype=c_double)
    mu = np.array([0 for i in range(len(b))],dtype=c_double)
    lam_star =  np.array([0 for i in range(len(b))],dtype=c_double)
    print("initial lambda", lam)
    print("initial working set", W)
    print("mu",mu)
    iter = 0
    while True:
        if iter == 10:
            print("break because of iterations")
            break
        iter += 1
        print("---------")
        print("Iteration",iter)
        W = np.sort(W)
        W = list(W)
        print("W",W, "len(W)", len(W))
        W_bar =  np.sort([x for x in range(len(lam)) if x not in W])
        print("W_bar", W_bar)
        mu = np.array([0 for i in range(len(b))],dtype=c_double)
        print("mu at the beginning of the round",mu)
        p = np.array([0 for i in range(len(b))],dtype=c_double)
        if len(W)==0 or is_invertible(M[W,:]@ M[W,:].T): # step 2
            print("case: MkMkT invertible")
            print("lam_star",lam_star)
            print("lam",lam)
            print("len(W)",len(W))
            if len(W)>0:
                print("M[W]",M[W,:])
                print("M[W] @M[W].T",M[W,:] @M[W,:].T)
                print("np.linalg.inv(M[W] @M[W].T)",np.linalg.inv(M[W] @M[W].T))
                print("d[W]",d[W],d[W].shape)
                print("- np.linalg.inv(M[W] @M[W].T) @d[W]",- np.linalg.inv(M[W] @M[W].T) @d[W])
                lam_star[W] = (- np.linalg.inv(M[W,:] @M[W,:].T) @d[W]) #.reshape(-1) # solve the system, step 3
            else:
                lam_star=np.array([0 for i in range(len(b))],dtype=c_double)
            print("lam",lam)
            print("lam_star",lam_star)
            if (np.array(lam_star) >= -1e-12).all(): # step 4
                print("case: all lam_star>=0")
                print("len(W), len(W_bar)",len(W), len(W_bar))
                print("mu before update", mu)
                if len(W)>0:
                    if len(W_bar)>0: # constraints are split between W and W_bar
                        print("M[W_bar]",M[W_bar,:])
                        print("M[W]",M[W,:])
                        print("lam_star[W]",lam_star[W])
                        print("d[W_bar]",d[W_bar])
                        print(" M[W_bar] @M[W].T", M[W_bar,:] @M[W,:].T)
                        print("M[W_bar] @M[W].T@lam_star[W]",M[W_bar,:] @M[W,:].T@lam_star[W])
                        print("M[W_bar] @M[W].T@lam_star[W]+d[W_bar]",M[W_bar,:] @M[W,:].T@lam_star[W]+d[W_bar])
                        mu[W_bar] = M[W_bar,:] @M[W,:].T@lam_star[W]+d[W_bar]
                    else: # all constraints in W
                        print("case: all constraints in W_bar")
                        mu = np.array([0 for i in range(len(b))],dtype=c_double)
                elif len(W)==0: # all constraints in W_bar
                    mu = np.copy(d)
                print("mu after update",mu)
                print("update lam now")
                print("mu",mu)
                lam = np.copy(lam_star)
                print("lam",lam)
                print("mu",mu)
                if (np.array(mu) >= -1e-6).all(): # step 6
                    # optimum found
                    print("case: all mu >=0")
                    lam_star_normalized = np.divide(lam_star, row_norm.reshape(-1), out=np.zeros_like(lam_star), where=row_norm.reshape(-1)!=0)
                    print("print all optimal values",R,M,lam_star_normalized, W,iter)
                    # problem: algorithm doesn't stop, when working set contains everything
                    print(lam_star.dtype)
                    print((lam_star/row_norm.reshape(-1)).dtype)
                    return -np.linalg.inv(R)@(M[W,:].T@lam_star[W]+v), lam_star_normalized, W, iter
                else: # step 7                  
                    print("case: not all mu >=0")
                    print("mu_W_bar",mu[W_bar])
                    j = np.argmin(mu[W_bar])
                    print("j",W_bar[j])
                    W.append(int(W_bar[j]))
                    print("new W for next round",W)
                    mu[j]=0     # was not there in the old version
            else:             
                print("case: lam_star<0") # step 8
                print("lam_star",lam_star)
                print("lam",lam)
                p = lam_star-lam # step 9
                print("p",p)
                B = [i for i in W if lam_star[i]<1e-12]
                print("B",B)
                lam,lam_star,mu, W = fix_component(lam,lam_star,mu, W, B, p) # step 10
                print("lm_star",lam_star)
        else:
            print("MkMkT not invertible")
            print("M[W]@ M[W].T",M[W]@ M[W].T)
            print("rank",np.linalg.matrix_rank(M[W,:]@ M[W,:].T))
            print("W",W)
            print("M[W]",M[W,:])
            MM=Matrix(M[W,:] @ M[W,:].T) # step 12
            MM = np.array(MM, dtype=np.float64)
            print("MM",MM)
            nullspace = scipy.linalg.null_space(MM)
            print("nullsapce",nullspace)
            print("length of MM nullspace",len(nullspace))
            if len(nullspace)==0:
                p[W] = np.zeros((1,len(W)))
            else:
                if nullspace.shape[1]>1:
                    p[W] = nullspace[0]
                    print("p",p.T)
                    print("d",d)
                    print("multiplication",p.T@d)
                    if p.T@d< 1e-12:
                        print("case: p.Td<0 holds")
                    else:
                        p = -p
                        
                    # for i in nullspace: #  (iterate over potential vectors in the nullspace)
                    #     print("element of nullspace",i)
                    #     pp = p
                    #     pp[W] = i
                    #     print("pp",pp.T)
                    #     print("d",d)
                    #     print("multiplication",pp.T@d)
                    #     if pp.T@d< 1e-12: 
                    #         print("case: test constraint") 
                    #         p[W] = i
                    #         break
                else:
                    p[W] = nullspace.reshape(-1)
                    print("p",p.T)
                    print("d",d)
                    print("multiplication",p.T@d)
                    if p.T@d< 1e-12:
                        print("case: p.Td<0 holds")
                    else:
                        p = -p
                    
            print("p",p)          
            B = [i for i in W if p[i]<1e-12] #step 13
            lam,lam_star,mu, W =  fix_component(lam,lam_star,mu,W,B,p) # step 14
            print("lam_star",lam_star)

           
x_star, lam_star, W, iter = daqp_self(H,ftot,A,btot,sense,[])  
print("final outcome", x_star, lam_star,W, "iterations", iter)

shape of M (5, 2)
row_norm [[17912.16401207]
 [ 8303.75354446]
 [37867.50487635]
 [21955.52327116]
 [21392.49682474]]
M before [[-2.38089742e-02 -1.79121640e+04]
 [-5.81989250e-01  8.30375352e+03]
 [-1.69626950e-01 -3.78675049e+04]
 [ 4.78774865e-02  2.19555233e+04]
 [-1.76239506e-01 -2.13924968e+04]]
M after [[-1.32920702e-06 -1.00000000e+00]
 [-7.00874908e-05  9.99999998e-01]
 [-4.47948578e-06 -1.00000000e+00]
 [ 2.18065795e-06  1.00000000e+00]
 [-8.23837943e-06 -1.00000000e+00]]
d before [-1.00792247e+09  4.67254530e+08 -2.13081508e+09  1.23544343e+09
 -1.20376177e+09]
d after [-56270.2792974   56270.27910186 -56270.2793388   56270.27937608
 -56270.27932788]
initial values
R [[ 1.72356754e+00 -1.30074734e+00]
 [ 0.00000000e+00  4.70345800e-05]]
M [[-1.32920702e-06 -1.00000000e+00]
 [-7.00874908e-05  9.99999998e-01]
 [-4.47948578e-06 -1.00000000e+00]
 [ 2.18065795e-06  1.00000000e+00]
 [-8.23837943e-06 -1.00000000e+00]]
v [1.12441924e+00 5.62702793e+04]
d [-56270.2792974   56270.2791

TypeError: cannot unpack non-iterable NoneType object

In [77]:
import self_implement_daqp  
import importlib  
importlib.reload(self_implement_daqp)  
from self_implement_daqp import daqp_self  

x_star, lam_star, W, iter = daqp_self(H,ftot,A,btot,sense,[])  
print("final outcome", x_star, lam_star,W, "iterations", iter)

shape of M (5, 2)
row_norm [[3.98402041]
 [0.90409067]
 [3.71481394]
 [2.38674009]
 [1.87489774]]
M before [[-3.1243222   2.47204963]
 [ 0.88662162 -0.17686731]
 [-3.03301847  2.14491061]
 [-2.36839723  0.29533509]
 [-0.45296625  1.81935788]]
M after [[-0.7842134   0.62049121]
 [ 0.98067777 -0.19563006]
 [-0.81646578  0.57739382]
 [-0.99231468  0.12373994]
 [-0.24159518  0.97037713]]
d before [-22.96011227   4.34810169 -20.57897562  -7.49516308 -12.2468688 ]
d after [-5.76305087  4.80936463 -5.53970561 -3.14033485 -6.53201961]
initial values
R [[0.68057382 1.37016768]
 [0.         1.63530283]]
M [[-0.7842134   0.62049121]
 [ 0.98067777 -0.19563006]
 [-0.81646578  0.57739382]
 [-0.99231468  0.12373994]
 [-0.24159518  0.97037713]]
v [ 4.76286157 -4.03358194]
d [-5.76305087  4.80936463 -5.53970561 -3.14033485 -6.53201961]
---------
initial lambda [0. 0. 0. 0. 0.]
initial working set []
mu [0. 0. 0. 0. 0.]
---------
Iteration 1
W [] len(W) 0
W_bar [0 1 2 3 4]
mu at the beginning of the rou